This notebook merge the data collected querying all the reviews and app details (with raitings) for both markets (Android and iOS)

In [1]:
import os
import json
import re
import csv
import pandas as pd

In [2]:
APP_LIST = 'dating-apps'
PROJECT = 'dating'
PATH_ANDROID = '/Users/pietro/Downloads/dl-dating/data/' + PROJECT + '/play-store/'
PATH_IOS = '/Users/pietro/Downloads/dl-dating/data/' + PROJECT + '/app-store/'
LIST = '/Users/pietro/Downloads/dl-dating/data/apps/' + APP_LIST + '.json'

In [3]:
with open(LIST) as json_file:
    APPS = json.load(json_file)

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
COLS = ["app", "lang", "reviews", "install", "minInstall", "maxInstall", "score", "ratings", "reviews"]
ANDROID_REVIEWS = pd.DataFrame(columns=COLS)

for app in APPS['apps']:
    app_name_dashed = app["android"]["id"].replace(".", "-")
    for subdir, dirs, files in os.walk(PATH_ANDROID + 'reviews/' + app_name_dashed):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']

        for fname in files:
            reviews = pd.read_csv(subdir + '/' + fname)
    
            app = os.path.basename(os.path.normpath(subdir))
            language =  fname.split(".")[0]

            try:
                fpath = PATH_ANDROID + 'details/' + app_name_dashed + '/'+ language + '.json'
                if os.path.isfile(fpath):
                    with open(fpath) as json_file:
                            data = json.load(json_file)
                else:
                    data = {'install': None, 'minInstalls': None, 'installs': None, 'score': None, 'ratings': None, 'reviews': None}  


                df = pd.DataFrame([[app, language, reviews.shape[0], data['installs'], data['minInstalls'], data['installs'], data['score'], data['ratings'], data['reviews']]], columns=COLS)
                ANDROID_REVIEWS = ANDROID_REVIEWS.append(df)
            except Exception as ex:
                print(f"error {ex}")
        
ANDROID_REVIEWS.to_csv(PATH_ANDROID + 'reviews-play-store.csv', index=False)

In [17]:
COLS = ["app", "country", "reviews", "ratings", "ratings_count"]
IOS_REVIEWS = pd.DataFrame(columns=COLS)

for app in APPS['apps']:
    for subdir, dirs, files in os.walk(PATH_IOS  + '/details/' + app['ios']['id']):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']
        
        for fname in files:
            country = fname.split(".")[0].upper()
            # reviews = pd.read_csv(subdir + '/' + fname)

            # app = os.path.basename(os.path.normpath(subdir))
            # country =  re.search('(.+?)(\.[^.]*$|$)', fname).group(1)
            
            try:
                if os.path.exists(f"{PATH_IOS}/reviews/{app['ios']['id']}/{country}.csv"):
                    reviews = pd.read_csv(f"{PATH_IOS}/reviews/{app['ios']['id']}/{country}.csv")
                else:
                    reviews = pd.DataFrame()
                    
                fpath = PATH_IOS + 'details/' + app['ios']['id'] + '/' + fname
                # if os.path.isfile(fpath):
                with open(fpath) as json_file:
                    data = json.load(json_file)[0]
                # else:
                #     data = {'appId': None, 'score': None, 'ratings': None, 'histogram': {}}

                
                n_reviews = -1 if reviews.empty else reviews.shape[0]
                df = pd.DataFrame([[app['ios']['id'], country, n_reviews, data['averageUserRating'], data['userRatingCount']]], 
                                  columns=COLS)
                
                IOS_REVIEWS = IOS_REVIEWS.append(df)
            except FileNotFoundError:
                data 
            # except Exception as ex:
            #     print(app, country)
            #     print(ex)
                #print(f"File not found {PATH_IOS + 'details/' + app + '-' + country.lower() + '.json'}")

IOS_REVIEWS.to_csv(PATH_IOS + 'reviews-app-store.csv', index=False)